We have to define the dressed Green's function, using the Dyson equation, and the vertex function given by the experimentally measured susceptibility.

Then we have to define the particle-particle vertex function, from the particle-hole vertex function.

With these ingredients, we now try out the solve_Eliashberg function, and plot the largest eigenvector.

For the first part, we need 
1) the bare Green's function
2) the susceptibility in imaginary frequency, to make a vertex
3) the self-energy from the vertex and the bare Green's function

Then we get the dressed Green's function from the Dyson equation.

In [1]:
from triqs.gf import *
from triqs.plot.mpl_interface import *
from triqs.lattice import *
from triqs.gf.tools import *
from triqs_tprf.lattice import lattice_dyson_g0_wk
from triqs_tprf.lattice import lattice_dyson_g_wk
# from triqs_tprf.lattice_utils import imtime_bubble_chi0_wk
from triqs_tprf.tight_binding import TBLattice
import numpy as np
from scipy.integrate import quad
import gc

pi = np.pi
g = 1.0


Starting serial run at: 2023-11-26 16:44:42.551712


The next step is to load the susceptibility data from the functional form in Dahm et al into Green's function form, and compute a vertex function from it, with a single free parameter - the coupling strength U. The vertex function is a Gf object with the same structure as bubble. 

In [2]:
n_w = 121
wmesh = MeshReFreq(window = (0.,120.), n_w = n_w)

BL = BravaisLattice([(1,0,0),(0,1,0)])
BZ = BrillouinZone(BL)
n_k = 16
kmesh = MeshBrZone(BZ, n_k = n_k)

kwmesh = MeshProduct(kmesh, wmesh)

In [3]:
# define functions to transform between k and HKL
def k_to_HKL(k):
    return k @ np.linalg.inv(kmesh.domain.units)
k_to_HKL([2*pi,0.,0.])
def HKL_to_k(HKL):
    return HKL @ kmesh.domain.units

In [4]:
# Now define a fit function for the susceptibility

import latexify
#constants
N0l = 0.001807; sigma_a = 0.125; sigma_b = 0.17; Q_0a = 0.5; Q_0b = 0.5+0.0001; #this is so that the lattice is not non-analytic at Q_0
Q_1a = 0.575; Q_1b = 0.5; Q_2a = 0.425; Q_2b = 0.5; Gamma = 11.0; p = 4; S_a = 4830.0; S_b = 10065.0; N0u = 1.35; w_dip = 47.0; sigma_dip = 2.0; N0 = 1.; Omega_r = 38.5; 
# N.B. S_a and S_b are defined in the paper as S_a = 4830.0 meV Angstrom^4 and S_b = 10065.0 meV Angstrom^4 respectively, but in our code we take the lattice constant to be unity
@latexify.function
def lower_fit_function(Qa, Qb, w):
    term1 = np.exp(-4 * np.log(2) * (((Qa - Q_1a) / sigma_a) ** 2 + ((Qb - Q_1b) / sigma_b) ** 2))
    term2 = np.exp(-4 * np.log(2) * (((Qa - Q_2a) / sigma_a) ** 2 + ((Qb - Q_2b) / sigma_b) ** 2))
    return N0 * N0l * Nlin(w) * Nl(w) * NL(w) * (term1 + term2)

def Nl(w):
    return 1 - 0.5/(1 + np.exp(w - 29.5)/2) #half ampltiude filter below 29.5 meV to half strength with a width of 2 meV

def NL(w):
    return 1/(1 + np.exp(w - 37)/2) #cut off at 37 meV with a width of 2 meV

def Nlin(w):
    if w >= 27:
        return 1
    else:
        return w/27
    
@latexify.function    
def upper_fit_function(Qa, Qb, w):
    return N0 * N0u * Nu(w) * Ndip(w) * Nrot(Qa,Qb) * (2*w*Gamma)/((w**2 - disp(Qa,Qb)**2)**2 + (2*w*Gamma)**2)

def disp(Qa, Qb):
    term1 = S_a**(2/p) * (Qa - Q_0a)**2
    term2 = S_b**(2/p) * (Qb - Q_0b)**2
    Omega_Q = Omega_r + (term1 + term2)**(p/2)
    return Omega_Q

def Nu(w):
    return 1 - 1.02/(1 + np.exp(w - 37)/2) #1 - 1/(1 + np.exp(w - 36)/1.5) #cut off below 36 meV with a width of 1.5 meV

def Ndip(w):
    return 1 #1 - 0.2 * np.exp(-(w-w_dip)**2 / (2 * sigma_dip**2))

def Nrot(Qa, Qb):
    return 1 - 0.3 * (((Qa - Q_0a)**2 - (Qb - Q_0b)**2) / ((Qa - Q_0a)**2 + (Qb - Q_0b)**2))**2

fit_function = np.vectorize(lambda Qa, Qb, w: lower_fit_function(Qa, Qb, w) + upper_fit_function(Qa, Qb, w))

# latexify.get_latex(upper_fit_function)

lower_fit_function
upper_fit_function
# try this later after restarting the kernel


In [5]:
# load the experimental data into chi_expt, a ReFreq Gf with target_shape = [] 
chi_expt = Gf(mesh = kwmesh, target_shape = [])

for k, w in kwmesh:
    k_HKL = k.value/(2 * pi) #k.value @ np.linalg.inv(kmesh.domain.units)
    chi_expt[k,w] = fit_function(k_HKL[0], k_HKL[1], w.value)

In [6]:
# Next setup a tight binding model for the cuprates, with nearest neighbour hopping t, next nearest neighbour hopping t', and next next nearest neighbour hopping t''. It must have two bands per spin, for bonding and antibonding states.

TA=409;TB=550;TPA=150;TPB=231;TPPA=40;TPPB=67;MUA=556;MUB=417
H = TBLattice(
    units = [(1, 0, 0), (0, 1, 0)],
    hopping = {
        # chemical potential
        ( 0, 0): np.diag([MUA,MUA,MUB,MUB]),
        # nearest neighbour hopping -t
        ( 0,+1): -np.diag([TA,TA,TB,TB]),
        ( 0,-1): -np.diag([TA,TA,TB,TB]),
        (+1, 0): -np.diag([TA,TA,TB,TB]),
        (-1, 0): -np.diag([TA,TA,TB,TB]),
        # next nearest neighbour hopping -t'
        (+1,+1): -np.diag([TPA,TPPA,TPB,TPPB]),
        (+1,-1): -np.diag([TPA,TPPA,TPB,TPPB]),
        (-1,+1): -np.diag([TPA,TPPA,TPB,TPPB]),
        (-1,-1): -np.diag([TPA,TPPA,TPB,TPPB]),
        # next next nearest neighbour hopping -t''
        (+2, 0): -np.diag([TPPA,TPPA,TPPB,TPPB]),
        (-2, 0): -np.diag([TPPA,TPPA,TPPB,TPPB]),
        ( 0,+2): -np.diag([TPPA,TPPA,TPPB,TPPB]),
        ( 0,-2): -np.diag([TPPA,TPPA,TPPB,TPPB]),
        },
    orbital_positions = [(0,0,0)]*4,
    orbital_names = ['up-bo', 'do-bo', 'up-ab', 'do-ab'],
    )
e_k = H.fourier(kmesh)

In [18]:
# First we need to define the bare Green's function for the tight binding model
K_B = 8.617333262145e-2 # Boltzmann constant in meV/K
T = 70 # Temperature in Kelvin
n_iw = 1200 # Number of positive Matsubara frequencies

iwmesh = MeshImFreq(beta=1.0/(K_B*T), S='Fermion', n_iw=n_iw)
kiwmesh = MeshProduct(kmesh, iwmesh)

ek=H.fourier(kmesh)
g0_iwk= lattice_dyson_g0_wk(mu=0.0, e_k=H.fourier(kmesh), mesh=iwmesh)

# G0_kiw = Gf(mesh=kiwmesh, target_shape=[4,4])
# for kid,k in enumerate(kmesh):
#     for wid, iw in enumerate(iwmesh):
#         G0_kiw.data[kid,wid,:,:] = np.linalg.inv(iw.value*np.eye(4) - H.fourier(k.value))



1\) the bare Green's function => g0_iwk ☑️

Now we move on to 2) the susceptibility in imaginary frequency


In [9]:
# load the experimental data into a Gf with target_shape = [] defined on kiOmesh
from tqdm import tqdm
iOmesh=MeshImFreq(beta=1.0/(K_B*T), S='Boson', n_max=n_iw+1)
kiOmesh = MeshProduct(kmesh, iOmesh)
chi_expt_kiO = Gf(mesh = kiOmesh, target_shape = [], name=r'$\chi_expt$')
relerr = Gf(mesh = kiOmesh, target_shape = [], name=r'$\delta\chi_expt$')


Careful: the next cell takes about 1 hr to run on my laptop (might take 10x more since n_iw=1200 instead of 120). Output is saved to file in the next cell.

In [10]:

#code to store the relative error of the integral in relerr and the value of the integral 
for k, iO in tqdm(kiOmesh):
    HKL= k_to_HKL(k.value)
    val, error = quad(lambda w: fit_function(HKL[0],HKL[1],w)/(iO.value-w)/pi,0,120,complex_func=True)
    chi_expt_kiO[k,iO] = val
    relerr[k,iO] = abs(error)/abs(val)
    

# for n in np.arange(-120, 121):
#     val,err=quad(lambda w: fit_function(0.0,0.5,w)/(2j*pi*K_B*T*n - w)/pi,0,120,complex_func=True)
#     relerr[120+n]=abs(err)/abs(val)

  0%|          | 0/18688 [00:00<?, ?it/s]

100%|██████████| 18688/18688 [03:57<00:00, 78.68it/s] 


In [11]:
from h5 import HDFArchive
with HDFArchive('chi_expt_niw36.h5', 'w') as f:
    f['chi_expt'] = chi_expt_kiO
    f['relerr'] = relerr
    

In [14]:
from h5 import HDFArchive
with HDFArchive('chi_expt.h5', 'r') as f:
    chi_expt_kiO << f['chi_expt'] 
    relerr << f['relerr']

In [10]:
# now we define the vertex function
# For convenience we define the Vertex function in the order used in triqs_tprf (time, space) not (space, time) which is callable
iOkmesh = MeshProduct(iOmesh, kmesh)
Vertex = Gf(mesh=iOkmesh, target_shape=[4,4,4,4])
Vertex.data[:] = np.einsum('Oq,ab,cd->qOabcd',chi_expt_kiO.data,np.eye(4),np.eye(4))
Vertex.data[:] = g**2 * Vertex.data

2\) the susceptibility => chi_expt_kiO(k,i0) ☑️

And the vertex function => Vertex(i0,k) related by

$V_{abcd}(i\Omega_n,k)= \sigma^0_{ab}\chi_{expt}(k,i\Omega_n)\sigma^0_{cd}$
☑️

Now we move on to 3) the self-energy from the vertex and the bare Green's function

$\Sigma_{dc} (i\omega_l,k) = \sum_{n,q} G_{ab}(i\omega_l-i\Omega_n,k-q) V_{abcd}(i\Omega_n,q)$

this is a convolution in w,k space, hence just a product in r,$\tau$ space.

$\Sigma_{dc} (\tau,r) = G_{ab}(\tau,r) V_{abcd}(\tau,r) = G_{aa}(\tau,r) \chi_{expt}(\tau,r) \delta_{cd}$



In [19]:
#define G_rtau - the iFFT of g0_iwk

#define the real space mesh corresponding to the k-mesh
rmesh = MeshCyclicLattice(BL, n_k)
taumesh = MeshImTime(beta=iwmesh.beta, S='Fermion', n_max=6*n_iw+1)

# for each k do the fourier transform to imaginary time
tails_container = [None]*(n_k*n_k)
for i, k in tqdm(enumerate(kmesh)):
    tails_container[i] = g0_iwk[:,k].fit_tail()
G_ktau = Gf(mesh=MeshProduct(kmesh, taumesh), target_shape=[4,4])
for i,k in tqdm(enumerate(kmesh)):
    G_ktau[k,:] << Fourier(g0_iwk[:,k], known_moments=np.array(tails_container[i][0]))

#for each tau do the fourier transform to real space
G_rtau = Gf(mesh=MeshProduct(rmesh,taumesh), target_shape=[4,4])
transformed_data = np.fft.ifft2(G_ktau.data.reshape(n_k, n_k, *G_ktau.data.shape[1:]), axes=(0, 1))
transformed_data = transformed_data.reshape(*G_ktau.data.shape)#.transpose(1,0,2,3)
G_rtau.data[:] = transformed_data

166it [00:00, 1645.99it/s]

256it [00:00, 1861.02it/s]
256it [00:01, 132.03it/s]


We really don't need Vertex at this point, we just need the susceptibility in r,$\tau$ space.

In [16]:
taumeshBoson=make_gf_from_fourier(Gf(mesh=iOmesh, data=chi_expt_kiO.data[0,:])).mesh
gc.collect()
chi_expt_ktau = Gf(mesh=MeshProduct(kmesh, taumeshBoson), target_shape=[])
for kid, k in tqdm(enumerate(kmesh)):
    chi_expt_ktau.data[kid,:] = make_gf_from_fourier(chi_expt_kiO[k,:]).data

256it [00:00, 723.21it/s]


In [18]:
#for each tau do the fourier transform to real space
chi_expt_rtau = Gf(mesh=MeshProduct(rmesh,taumeshBoson), target_shape=[])
transformed_data = np.fft.ifft2(chi_expt_ktau.data.reshape(n_k, n_k, *chi_expt_ktau.data.shape[1:]), axes=(0, 1))
transformed_data = transformed_data.reshape(*chi_expt_ktau.data.shape)#.transpose(1,0,2,3,4,5)
chi_expt_rtau.data[:] = transformed_data

In [51]:
chi_expt_rtau((0,0,0),0.1)

(-7.431123377558212e-05+4.0674064886034536e-13j)

In [66]:
SelfEn_rtau = Gf(mesh=MeshProduct(rmesh,taumesh), target_shape=[4,4])
gc.collect()
for r in tqdm(rmesh):
    rtuple = tuple(r.value.astype(int))
    for tau in taumesh:
        for i in range(4):
             SelfEn_rtau[r,tau][:,:] += np.eye(4)* (G_rtau[i,i](rtuple,tau) * chi_expt_rtau(rtuple,tau))


100%|██████████| 256/256 [08:20<00:00,  1.96s/it]


In [67]:
from h5 import HDFArchive
with HDFArchive('SelfEn.h5', 'w') as f:
    f['SelfEn_rtau'] = SelfEn_rtau
# maybe store only the diagonal elements of SelfEn_rtau, with four times less space 

In [68]:
# now fourier transform to get the self energy in momentum and frequency space
SelfEn_ktau = Gf(mesh=MeshProduct(kmesh, taumesh), target_shape=[4,4])
transformed_data = np.fft.fft2(SelfEn_rtau.data.reshape(n_k, n_k, *SelfEn_rtau.data.shape[1:]), axes=(0, 1))
transformed_data = transformed_data.reshape(*SelfEn_rtau.data.shape)
SelfEn_ktau.data[:] = transformed_data

In [78]:
SelfEn_kiw = Gf(mesh=kiwmesh, target_shape=[4,4])
for k in tqdm(kmesh):
    SelfEn_kiw[k,:] << Fourier(SelfEn_ktau[k,:])

100%|██████████| 256/256 [00:02<00:00, 119.59it/s]


In [82]:
G_kiw = Gf(mesh=kiwmesh, target_shape=[4,4])
for ik,k in enumerate(kmesh):
    for niw, iw in enumerate(iwmesh):
        G_kiw.data[ik,niw,:,:] = np.linalg.inv(np.linalg.inv(g0_iwk.data[niw,ik,:,:]) - SelfEn_kiw.data[ik,niw,:,:])

In [84]:
#Finally store the dressed Green's function to file
from h5 import HDFArchive
with HDFArchive('Gkiw.h5', 'w') as f:
    f['G_kiw'] = G_kiw

: 